In [ ]:
"""
Purpose: To check on the status of the Allen split suggestions pipeline
as they run through

- Downloading
- Decimation
- Soma Finding
- Decomposition
- Suggestion Splits

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-04-15 20:01:19,002 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-15 20:01:19,004 - settings - Setting database.user to celiib
INFO - 2021-04-15 20:01:19,005 - settings - Setting database.password to newceliipass
INFO - 2021-04-15 20:01:19,021 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-15 20:01:19,021 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-15 20:01:19,032 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-15 20:01:19,365 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-15 20:01:19,368 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-15 20:01:19,369 - settings - Setting database.user to celiib
INFO - 2021-04-15 20:01:19,369 - settings - Setting database.password to newceliipass
INFO - 2021-04-15 20:01:19,369 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-15 20:01:19,370 - settings - Setting database.user to celiib
INFO - 2021-04-15 20:01:19,370 - settings - Setting database.password to newceliipass
INFO - 2021-04-15 20:01:19,371 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-15 20:01:19,736 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-15 20:01:19,842 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-15 20:01:19,843 - settings - Setting database.user to celiib
INFO - 2021-04-15 20:01:19,844 - settings - Setting database.password to newceliipass
INFO - 2021-04-15 20:01:19,845 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-15 20:01:20,255 - settings - Setting enable_python_native_blobs to True


# Investigating why some neurons did not get split suggestions

In [5]:
allen_table = (minnie.AllenProofreading() & dict(month=4,day=14,year=2021))
allen_table

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691135337768422,4,14,2021,2
864691135373423816,4,14,2021,2
864691135373605832,4,14,2021,2
864691135385570133,4,14,2021,2
864691135493989520,4,14,2021,2
864691135569218694,4,14,2021,2
864691135639122747,4,14,2021,2
864691135657789570,4,14,2021,2
864691135748641321,4,14,2021,2
864691135758712910,4,14,2021,2


In [6]:
minnie.BaylorSegmentCentroid() & allen_table  & dict(decimation_ratio=0.25) & "multiplicity=0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)


In [7]:
key_source = minnie.Decomposition() & (allen_table).proj()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135337768422,0,0.25,30.00,5,0,1,=BLOB=,693597,1426132,0,1,1,1,2,14,124,76,4980587.301357684,2601189.828897018,24641.769,357.039,321.363,652.032,650.889,2787,0.0005595725627056627,22.475806451612904,4824585.191,117,0.0005776662427267315,23.821,468044957417.12427,113483413.69298425,93973.84868437855,97012.47649025591,4000384251.4284124,2714.5003
864691135373423816,0,0.25,30.00,4,0,1,=BLOB=,3383405,6837747,0,0,0,0,1,8,853,808,29828707.39402775,27323282.569861326,25105.522,83.504,83.504,231.078,231.078,359,1.2035385752983661e-05,0.4208675263774912,8650394.095,265,4.1500999383080705e-05,1.355,59226635728.49248,77291152.34234586,1985.5582391193634,6846.697974457136,223496738.5980848,8559.7473
864691135373605832,0,0.25,30.00,6,0,1,=BLOB=,1754675,3531550,0,1,1,1,2,13,974,910,30822302.69561133,27454940.78360191,22357.16,85.461,85.461,221.207,215.774,2065,6.699694115631494e-05,2.120123203285421,8739432.332,272,0.00023628536975323536,7.592,409531910597.21326,95758495.7097168,13286.869402380014,46860.24160833484,1505632024.2544606,4083.9395
864691135385570133,0,0.25,30.00,6,0,1,=BLOB=,697273,1436366,0,1,1,1,2,14,207,138,10190348.208967423,5725229.38384963,30169.436,84.377,84.377,454.686,449.536,1303,0.00012786609184300206,6.294685990338165,5115542.123,82,0.0002547139616232616,15.89,212143062183.32126,109360435.29699707,20818.038582492904,41470.29915549017,2587110514.430747,2136.2261
864691135493989520,0,0.25,30.00,0,0,1,=BLOB=,903491,1834683,0,1,0,1,2,14,179,74,7803189.876187458,3282828.3573968434,30943.038,339.008,268.449,517.829,467.959,5163,0.0006616524885233958,28.843575418994412,7088379.964,156,0.0007283751754592031,33.096,767017120574.7532,85428788.12639236,98295.32956969494,108207.67572706733,4916776413.940725,3298.3634
864691135569218694,0,0.25,30.00,6,0,1,=BLOB=,1116988,2241669,0,1,1,1,2,10,402,336,13363544.081041824,9769926.197946006,19199.655,9

In [8]:
key_source & "n_multi_soma_touching_limbs = 0 AND n_same_soma_multi_touching_limbs=0 "

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135373423816,0,0.25,30.00,4,0,1,=BLOB=,3383405,6837747,0,0,0,0,1,8,853,808,29828707.39402775,27323282.569861326,25105.522,83.504,83.504,231.078,231.078,359,1.2035385752983661e-05,0.4208675263774912,8650394.095,265,4.1500999383080705e-05,1.355,59226635728.49248,77291152.34234586,1985.5582391193634,6846.697974457136,223496738.5980848,8559.7473
864691135873745550,0,0.25,30.00,6,0,1,=BLOB=,687513,1367618,0,0,0,0,1,14,162,52,6542834.053294278,1386749.5798392799,22092.927,275.717,266.876,491.372,454.239,2357,0.0003602414459546417,14.549382716049383,5340548.035,107,0.0004413404737778002,22.028,525897372545.9979,121363728.53999248,80377.61133208197,98472.54796688631,4914928715.383158,2556.0695
864691137197156929,0,0.25,30.00,6,0,1,=BLOB=,942154,1919892,0,0,0,0,2,16,258,99,9091416.081089724,2995196.5220857547,22321.958,201.21,198.497,496.335,475.772,3211,0.0003531903029582956,12.445736434108527,5839296.142,143,0.0005498950424700005,22.455,725512709265.7205,125683412.6983579,79801.94755081084,124246.60295054452,5073515449.410632,4469.0943


In [ ]:
import neuron_utils as nru

In [ ]:
segment_id = 864691135782479155
dec_mesh = du.fetch_segment_id_mesh(segment_id)
neuron_obj = nru.decompress_neuron("/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135782479155_0_25.pbz2",
                     original_mesh=dec_mesh)

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
neuron_obj.n_error_limbs

In [ ]:
nviz.visualize_neuron_lite(neuron_obj)

In [ ]:
segment_id_with_no_errors = [
    864691135345291423, #had just soma on the very tip
    864691135373423816, #removed in glia
    864691135753669837,
    ,
]

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'") & "status='error'"
(curr_table).delete()# & "status='error'")

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
seg_ids_to_delete = [k["segment_id"] for k in curr_table.fetch("key")]
seg_ids_to_delete

In [ ]:
minnie.Decomposition()

# All the cells that need to get suggestions

In [ ]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

In [ ]:
no_splits = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)) - key_source.proj()
no_splits

In [ ]:
minnie.DecompositionSplit() & key_source.proj()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")

# Getting the split suggesstions

In [10]:
import datajoint as dj
dj.U("segment_id") & minnie.DecompositionSplit() & key_source.proj()

segment_id segment id unique within each Segmentation
864691135337768422
864691135385570133
864691135493989520
864691135569218694
864691135639122747
864691135657789570
864691135748641321
864691135758712910
864691135777371837
864691135864237182


In [11]:
allen_segment_ids = (minnie.AllenProofreading() & dict(day=14,month=4,year=2021)).fetch("segment_id")
allen_segment_ids

array([864691135337768422, 864691135373423816, 864691135373605832,
       864691135385570133, 864691135493989520, 864691135569218694,
       864691135639122747, 864691135657789570, 864691135748641321,
       864691135758712910, 864691135777371837, 864691135864237182,
       864691135866790422, 864691135866807318, 864691135866815254,
       864691135873745550, 864691136091193268, 864691136099972213,
       864691136174990854, 864691136194189270, 864691137197156929,
       864691137197169985])

In [13]:
allen_segment_ids = (allen_table).fetch("segment_id")
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=allen_segment_ids,output_type="server",
                                                   print_links=True,
                                                   output_filename="allen_spreadsheet_4_14.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

segment_id = 864691135337768422
segment_id = 864691135385570133
segment_id = 864691135493989520
segment_id = 864691135569218694
segment_id = 864691135639122747
segment_id = 864691135657789570
segment_id = 864691135748641321
segment_id = 864691135758712910
segment_id = 864691135777371837
segment_id = 864691135864237182
segment_id = 864691135866790422
segment_id = 864691135866807318
segment_id = 864691135866815254
segment_id = 864691136099972213
segment_id = 864691136174990854
segment_id = 864691136194189270
segment_id = 864691137197156929
segment_id = 864691137197169985
864691135337768422: http://neuromancer-seung-import.appspot.com/?json_url=https://globalv1.daf-apis.com/nglstate/api/v1/6445814566092800
864691135385570133: http://neuromancer-seung-import.appspot.com/?json_url=https://globalv1.daf-apis.com/nglstate/api/v1/4991388517138432
864691135493989520: http://neuromancer-seung-import.appspot.com/?json_url=https://globalv1.daf-apis.com/nglstate/api/v1/6285994370269184
8646911355692